# 1. Importations

In [ ]:
import pandas as pd
import warnings
import numpy as np
import seaborn as sns
warnings.filterwarnings('ignore')
import random
from wordcloud import WordCloud
import matplotlib.pyplot as plt

# Charger le modèle spaCy
import spacy
# spacy.cli.download("en_core_web_md")
import en_core_web_md

from collections import Counter

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import TruncatedSVD, LatentDirichletAllocation
from sklearn.cluster import KMeans
from sklearn.manifold import TSNE

In [ ]:
chemin_fichier = 'reviews_processed_with_spacy_md.csv'
df = pd.read_csv(chemin_fichier)


# 2. Définition des fonctions

In [ ]:
# Chargement du dictionnaire
nlp = spacy.load("en_core_web_md")

In [ ]:
# Réduction du dataframe
col = ["firm","headline_clean","pros_clean","cons_clean"]
df_reduced = df[col].astype(str)
list_firms = df_reduced["firm"].unique()
# list_firms = ["Apple"]



In [ ]:
# Creation des fonctions
   
def process_wordcloud(df, custom_stopwords={}):
    """
    Fonction calculant et retournant le nuage de mots.
    """

    stop_words = set(nlp.Defaults.stop_words) # Utilisez un ensemble pour une recherche plus rapide

    if custom_stopwords:
        if isinstance(custom_stopwords, list):
            stop_words.update(custom_stopwords) # Ajouter les mots à l'ensemble
        else: #Si ce n'est pas une liste, on suppose que c'est déjà un set.
            stop_words.update(custom_stopwords)

    # Generate word cloud
    wordcloud = WordCloud(max_words = 30, width=1000, height=600, background_color='white', stopwords=stop_words).generate(' '.join(df.fillna('').astype(str)))


    return wordcloud

def display_wordcloud(wordcloud, main_subject_title):
    """
    Fonction affichant le nuage de mots.
    """
    # Display the word cloud
    plt.figure(figsize=(10, 5))
    plt.imshow(wordcloud, interpolation='bilinear')
    plt.axis('off')
    plt.title(f'Most Frequent Words in {main_subject_title} Reviews', fontsize=16, pad=40)
    plt.show()

def most_common_words_identification(df, nb_words=30):
    """
    Fonction calculant et retournant les mots les plus fréquents.
    """
    # Combine all pros texts into a single string
    text = ' '.join(df)

    # Split to wordsand count frequency
    word_counts = Counter(text.split())

    # Nombre of words
    total_word = len(text.split())

    # Checking top-nb_words
    common_words = word_counts.most_common(nb_words)
    # for word, count in common_words:
    #     print(f"{word}: {count} ({count / total_word:.2%})")
    return common_words


# Vectorize the pros column
def generate_vectorizer(df, n_gram_range=(1, 2), custom_stop_word={}):
    """
    Fonction pour générer le TF-IDF.
    """
    text = []

    stop_words = set(nlp.Defaults.stop_words) # Utilisez un ensemble pour une recherche plus rapide

    if custom_stop_word:
        if isinstance(custom_stop_word, list):
            stop_words.update(custom_stop_word) # Ajouter les mots à l'ensemble
        else: #Si ce n'est pas une liste, on suppose que c'est déjà un set.
            stop_words.update(custom_stop_word)

    for sentence in df:
        t = ' '.join([word for word in sentence.split() if word not in stop_words])
        text.append(t)

    vectorizer_TFIDF = TfidfVectorizer(max_features=1000, ngram_range=n_gram_range, stop_words='english', min_df = 10, max_df = 0.9)
    vectorizer_Count = CountVectorizer(max_features=1000, ngram_range=n_gram_range, stop_words='english', min_df = 10, max_df = 0.9)

    X_Tfidf = vectorizer_TFIDF.fit_transform(text)
    X_Count = vectorizer_Count.fit_transform(text)
    return X_Tfidf, X_Count, vectorizer_TFIDF,vectorizer_Count

def display_top_topics(vectorizer, model, n_top_words=10):
    """
    Fonction pour afficher les topics les plus fréquents.
    """
    terms = vectorizer.get_feature_names_out()
    # print("Terms:\n")
    # print(terms)
    for i, comp in enumerate(model.components_):
        terms_in_topic = [terms[j] for j in comp.argsort()[:-(n_top_words+1):-1]]
        print(f"Topic {i+1}: {' | '.join(terms_in_topic)}")

# SVD decomposition
def svd_decomposition(X, n_components=2):
    """
    Fonction pour effectuer la décomposition tronquée SVD.
    """
    svd = TruncatedSVD(n_components=n_components, random_state=42)
    # Fit the model and transform the data
    lsa = svd.fit(X)
    return lsa

def lda_decomposition(X,n_components=2):
    lda = LatentDirichletAllocation(n_components=n_components, random_state=42)
    return lda.fit(X)
 


In [ ]:
# --- Fonctions pour le traitement des lemmes (sous forme de chaînes) ---

def get_embedding_from_lemmas_string(lemmas_string, nlp_model):
    """
    Calcule l'embedding moyen à partir d'une chaîne de caractères contenant des lemmes séparés par des espaces.
    """
    lemmas_list = lemmas_string.split() # Fendre la chaîne en une liste de lemmes
    vectors = []
    for lemma in lemmas_list:
        # Vérifiez si le lemme est dans le vocabulaire du modèle et a un vecteur
        if lemma in nlp_model.vocab and nlp_model.vocab[lemma].has_vector:
            vectors.append(nlp_model.vocab[lemma].vector)

    if vectors:
        return np.mean(vectors, axis=0)
    else:
        # Retourne un vecteur de zéros si aucun lemme n'est trouvé ou n'a de vecteur
        return np.zeros(nlp_model.vocab.vectors.shape[1])


def process_lemmas_strings_in_batches(data_input_series, batch_size, nlp_model):
    """
    Fonction principale pour traiter une Series de chaînes de lemmes par lots et générer les embeddings.

    Args:
        data_input_series (pd.Series): Une Series pandas où chaque entrée est une chaîne de lemmes
                                       séparés par des espaces.
                                       Ex: pd.Series(["flexibilité horaire", "télétravail encourager"])
        batch_size (int): La taille des lots pour le traitement.
        nlp_model: L'instance du modèle spaCy chargé.

    Returns:
        tuple: Un tuple contenant (document_embeddings, original_statements_for_display)
    """
    document_embeddings = []
    original_statements_for_display = [] # Pour conserver la chaîne originale pour l'affichage

    if not isinstance(data_input_series, pd.Series):
        raise TypeError("data_input_series doit être une Series pandas.")

    # print(f"Génération des embeddings par lots (taille de lot : {batch_size})...")

    # Itérer sur les indices pour créer des lots à partir de la Series
    for i in range(0, len(data_input_series), batch_size):
        batch_of_strings = data_input_series.iloc[i:i + batch_size]

        for lemmas_string in batch_of_strings: # Chaque 'lemmas_string' est une chaîne d'un avantage
            # Vérifiez si la chaîne n'est pas vide (peut arriver après le nettoyage)
            if pd.isna(lemmas_string) or not lemmas_string.strip():
                embedding = np.zeros(nlp_model.vocab.vectors.shape[1])
            else:
                embedding = get_embedding_from_lemmas_string(str(lemmas_string), nlp_model) # Assurez-vous que c'est une str

            document_embeddings.append(embedding)
            original_statements_for_display.append(str(lemmas_string)) # Conserver la chaîne originale

    document_embeddings = np.array(document_embeddings)

    # print(f"Forme des embeddings de documents : {document_embeddings.shape}")


    return document_embeddings, original_statements_for_display



def k_means_algorithm(df,nb_clusters = 5):
    # --- Exécution du pipeline avec la Series de lemmes ---
    batch_size_for_processing = 1000

    document_embeddings, original_statements_for_display = process_lemmas_strings_in_batches(
        df, batch_size_for_processing, nlp
    )

    ## Application de l'algorithme K-Means
    n_clusters = nb_clusters # Nombre de catégories 

    if n_clusters > len(document_embeddings):
        n_clusters = len(document_embeddings)
        print(f"Attention : Le nombre de clusters a été ajusté à {n_clusters} car il y a moins de documents.")

    print(f"\nExécution de K-Means avec {n_clusters} clusters...")
    kmeans_model = KMeans(n_clusters=n_clusters, random_state=42, n_init=10)
    clusters = kmeans_model.fit_predict(document_embeddings)
    print("Clustering K-Means terminé.")

    ## Interprétation des clusters
    print("\n--- Échantillon de chaque cluster pour interprétation ---")
    df_results = pd.DataFrame({
        'statement': original_statements_for_display,
        'cluster': clusters,
        'embedding': list(document_embeddings)
    })

    for cluster_id in range(n_clusters):
        cluster_sample = df_results[df_results['cluster'] == cluster_id].sample(min(10, len(df_results[df_results['cluster'] == cluster_id])))
        print(f"Cluster {cluster_id} ({len(df_results[df_results['cluster'] == cluster_id])})")
        # for i, row in cluster_sample.iterrows():
        #     print(f"  - {row['statement']}")

    ## Visualisation des clusters (avec t-SNE)
    # t-SNE : algorithme de réduction de dimensionnalité non linéaire 
    # qui est particulièrement bien adapté à la visualisation de données de haute dimension (comme ici avec la taille des pros et cons)
    sample_size_for_tsne = min(2000, len(document_embeddings))
    sample_indices = random.sample(range(len(document_embeddings)), sample_size_for_tsne)
    sampled_embeddings = document_embeddings[sample_indices]
    sampled_clusters = clusters[sample_indices]
    sampled_statements = [original_statements_for_display[i] for i in sample_indices]

    if len(sampled_embeddings) > 5:
        print(f"\nRéduction de dimension avec t-SNE sur {len(sampled_embeddings)} échantillons (peut prendre du temps)...")
        tsne = TSNE(n_components=2, random_state=42, perplexity=min(30, len(sampled_embeddings) - 1), n_iter=1000)
        reduced_embeddings = tsne.fit_transform(sampled_embeddings)

        plt.figure(figsize=(12, 10))
        sns.scatterplot(
            x=reduced_embeddings[:, 0],
            y=reduced_embeddings[:, 1],
            hue=sampled_clusters,
            palette=sns.color_palette("hsv", n_clusters),
            legend="full",
            s=80,
            alpha=0.7
        )

        for i in random.sample(range(len(sampled_embeddings)), min(50, len(sampled_embeddings))):
            plt.text(reduced_embeddings[i, 0] + 0.05, reduced_embeddings[i, 1] + 0.05,
                    f"{i}: {sampled_statements[i][:20]}...", fontsize=6, alpha=0.6)

        plt.title(f"Visualisation des clusters (t-SNE - Échantillon de {len(sampled_embeddings)})")
        plt.xlabel("Composante t-SNE 1")
        plt.ylabel("Composante t-SNE 2")
        plt.grid(True)
        plt.show()
    else:
        print("\nPas assez de documents pour la visualisation t-SNE de l'échantillon.")


    return df_results

# 3. Traitements

## 3.1 Wordcloud

In [ ]:
#Common word for the whole dataset
nb_top_words = 10
common_word_pros = most_common_words_identification(df_reduced["pros_clean"], nb_top_words)
common_word_cons = most_common_words_identification(df_reduced["cons_clean"], nb_top_words)
common_word_headline = most_common_words_identification(df_reduced["headline_clean"], nb_top_words)
common_word = common_word_pros+common_word_cons+common_word_headline

In [ ]:
# Affichage des wordclouds
for firm in list_firms:
  df_firm = df_reduced[df_reduced["firm"] == firm].astype(str)
  print(f"Firm: {firm}\n")

  for c in ["pros","cons","headline"]:
      print(c.upper())
      custom_stop_word = {firm.lower()}
      custom_stop_word.update({word for word,count in common_word})
      if c == "pros":
         col = "pros_clean"
         custom_stop_word.update({word for word,count in common_word_pros})
      if c == "cons":
         col = "cons_clean"
         custom_stop_word.update({word for word,count in common_word_cons})
      if c == "headline":
         col = "headline_clean"
         custom_stop_word.update({word for word,count in common_word_headline})
      
      # Common words identification
      common_words = most_common_words_identification(df_firm[f"{col}"], 10)
      custom_stop_word.update({word for word,count in common_words})

            
      # WORDCLOUD
      wd = process_wordcloud(df_firm[f"{col}"], custom_stopwords=custom_stop_word)    
      if c == "headline":
        display_wordcloud(wd, f"{firm}: Headline")
      elif c == "pros":
        display_wordcloud(wd, f"{firm}: Pros")
      elif c == "cons":
        display_wordcloud(wd, f"{firm}: Cons")
      print()


## 3.2 Topics

In [ ]:
#  Affichage des topics
for firm in list_firms:
  df_firm = df_reduced[df_reduced["firm"] == firm].astype(str)
  print(f"Firm: {firm}\n")
  

#   for c in ["pros"]:
  for c in ["pros","cons","headline"]:
      print(c.upper())
      custom_stop_word = {firm.lower()}
      if c == "pros":
         col = "pros_clean"
         custom_stop_word.update({word for word,count in common_word_pros})
      if c == "cons":
         col = "cons_clean"
         custom_stop_word.update({word for word,count in common_word_cons})
      if c == "headline":
         col = "headline_clean"
         custom_stop_word.update({word for word,count in common_word_headline})
      
      # Common words identification
      common_words = most_common_words_identification(df_firm[f"{col}"], 10)
      custom_stop_word.update({word for word,count in common_words})

      # TOPICS
      # Generate TF-IDF matrix
      X_Tfidf ,X_Count, vectorizer_TFIDF, vectorizer_Count = generate_vectorizer(df_firm[f"{col}"],
                                                                               n_gram_range=(2, 3),
                                                                               custom_stop_word=custom_stop_word)
      
      nb_topics = 3
      # Perform LSA
      lsa = svd_decomposition(X_Tfidf, n_components=nb_topics)
      # Display top topics
      print(f"Top topics from LSA for {firm} in {c}:")
      display_top_topics(vectorizer_TFIDF, lsa, n_top_words=5)
      print()

      # # Perform LDA
      # lda = lda_decomposition(X_Count, n_components=nb_topics)
      # print(f"Top topics from LDA for {firm} in {c}:")
      # display_top_topics(vectorizer_Count, lda, n_top_words=5)
      # print()

      feature_names = vectorizer_Count.get_feature_names_out()
      word_counts = np.asarray(X_Count.sum(axis=0)).flatten() # Convertir la matrice somme en un array 1D

      # Créer un DataFrame Pandas pour faciliter le tri
      words_df = pd.DataFrame({'term': feature_names, 'count': word_counts})

      # 3.3. Trier par fréquence et sélectionner le top 3
      top_words = words_df.sort_values(by='count', ascending=False).reset_index().head(3)
      top_words = top_words.drop('index', axis=1)
      print("\n--- Top 3 des mots/n-grammes les plus utilisés dans le corpus ---")
      print(top_words)
      print("*********************")   

## 3.3 Clustering

In [ ]:
import re
nb_cluster = 9

for firm in list_firms:
    df_firm = df[df["firm"] == firm].astype(str)
    print(f"Firm: {firm}\n")

    # Étape 1: Remplacer les NaN par des chaînes vides pour éviter les erreurs lors du split
    # et s'assurer que c'est bien une chaîne de caractères
    df_firm['pros_processed'] = df_firm['pros_clean'].astype(str).replace('nan', '')

    # Étape 2: Diviser la chaîne par le point ('.') et la virgule (',').
    df_firm['pros_processed'] = df_firm['pros_processed'].apply(
        lambda x: [s.strip() for s in re.split(r'[.,]', x) if s.strip()]
    )

    # Étape 3: Utiliser .explode() pour transformer chaque élément de la liste en une nouvelle ligne.
    # Cela va dupliquer les autres colonnes (comme 'id_employee') pour chaque nouvelle entrée.
    df_expanded = df_firm.explode('pros_processed')

    # Étape 4: Nettoyer les lignes résultantes qui pourraient être vides
    df_expanded = df_expanded[df_expanded['pros_processed'] != ''].reset_index(drop=True)      

    # Étape 5: Application du clustering
    df_result = k_means_algorithm(df_expanded["pros_processed"], nb_cluster)
      
    most_common_words_identification(df_result[df_result["cluster"] == 0], 10)





In [ ]:
# result = most_common_words_identification(df_result[df_result["cluster"] == 0], 10)
# result
df_result